metadata can be found for after 2021 http://geneontology.org/docs/go-annotation-file-gaf-format-2.2/

metadata can be found for before 2021 http://geneontology.org/docs/go-annotation-file-gaf-format-2.1/

metadata can be found for before 2021 http://geneontology.org/docs/go-annotation-file-gaf-format-2.0/

prefixes can be found https://www.ebi.ac.uk/ols/index and http://geneontology.org/docs/sparql

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%%capture
!pip install obonet

In [2]:
import  requests
from    urllib.request import urlretrieve
import  os
import  re
import  pandas as pd 
import  numpy as np

In [3]:
os.chdir('/content/drive/MyDrive/GOA_Dataset/')

In [5]:
from src import utils

In [4]:
#Gdrive paths for saving train,test,valid sets
ONTO_GDRIVE_PATH = "/content/drive/MyDrive/GOA_Dataset/GO Versions/gene_ontology_"
GOA_DRIVE_PATH   = "/content/drive/MyDrive/GOA_Dataset/GO_Chicken/goa_chicken-"

SAVE_PATH        "/content/drive/MyDrive/GOA_Dataset/"

In [6]:
spec_version_v1     = "2023-32-32"
spec_version_v2     = "2023-35-35"

# SCENARIO 1: Only newly added annotations in test and valid sets

In [7]:
#get 2 GO human annotation version from 2017 to current (2022) each time. Now  "2017-01-01" and "2018-01-01"
#first version will be used for train set
triple_df = utils.generate_goa_graph(ONTO_GDRIVE_PATH, GOA_DRIVE_PATH, spec_version_v1,1)
triple_df.rename({0: 'head', 1: 'relation', 2:'tail'}, axis=1, inplace=True)
triple_df.head()

,head,relation,tail
0,https://www.uniprot.org/uniprot/X1,http://purl.obolibrary.org/obo/GO_0003674,http://purl.obolibrary.org/obo/g8
1,https://www.uniprot.org/uniprot/X2,http://purl.obolibrary.org/obo/GO_0005575,http://purl.obolibrary.org/obo/g9
2,https://www.uniprot.org/uniprot/X3,http://purl.obolibrary.org/obo/GO_0008150,http://purl.obolibrary.org/obo/g5
3,https://www.uniprot.org/uniprot/X4,http://purl.obolibrary.org/obo/GO_0008150,http://purl.obolibrary.org/obo/g4
4,http://purl.obolibrary.org/obo/g8,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://purl.obolibrary.org/obo/g4


In [8]:
#save train set to Drive folder
triple_df.to_csv(SAVE_PATH+'train.txt', sep='\t', index=False,header = False)

In [9]:
#take entities in V1
entities_v1 = utils.get_entities(triple_df)

In [10]:
#take V2 graph
second_df = utils.generate_goa_graph(ONTO_GDRIVE_PATH,GOA_DRIVE_PATH, spec_version_v2,0)
second_df.rename({0: 'head', 1: 'relation', 2:'tail'}, axis=1, inplace=True)
second_df.head()

,head,relation,tail
0,https://www.uniprot.org/uniprot/X1,http://purl.obolibrary.org/obo/GO_0003674,http://purl.obolibrary.org/obo/g8
1,https://www.uniprot.org/uniprot/X2,http://purl.obolibrary.org/obo/GO_0005575,http://purl.obolibrary.org/obo/g9
2,https://www.uniprot.org/uniprot/X3,http://purl.obolibrary.org/obo/GO_0008150,http://purl.obolibrary.org/obo/g8
3,https://www.uniprot.org/uniprot/X4,http://purl.obolibrary.org/obo/GO_0008150,http://purl.obolibrary.org/obo/g5


In [11]:
#define diff for test and valid sets
#diff between first and second versions will be used for test and valid set
diffv2_v1_ex  = utils.generate_test_valid(triple_df,second_df, entities_v1)
df_test, df_valid = utils.split_test_valid(diffv2_v1_ex)
#generate txt files
utils.txt_gen(SAVE_PATH+'test_sc1.txt',df_test)
utils.txt_gen(SAVE_PATH+'valid_sc1.txt',df_valid)

V2 different triple count:  2
different triples with existed entities 2


/content/drive/MyDrive/GOA_Dataset/src/utils.py:146: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ).query('_merge == "left_only"').drop('_merge', 1)


,0,1,2
0,https://www.uniprot.org/uniprot/X3,http://purl.obolibrary.org/obo/GO_0008150,http://purl.obolibrary.org/obo/g8


# SCENARIO 2: novel annotations + implict annotation (from True path rule)

In [13]:
#test and valid sets now contain data that fits TPR
#train set will remain the same
triple_df = utils.generate_goa_graph(ONTO_GDRIVE_PATH,GOA_DRIVE_PATH,spec_version_v1,0)
second_df = utils.generate_goa_graph(ONTO_GDRIVE_PATH,GOA_DRIVE_PATH,spec_version_v2,0)
#read gene ontology
graph_go = utils.read_gene_ontology(ONTO_GDRIVE_PATH+spec_version_v1)

In [14]:
#take entities in V1
entities_v1 = utils.get_entities(triple_df)

**First test and valid set case:** Level 1 hierarchical depth of implicit annotations for test and valid sets

In [15]:
#Level 1: Use levels as hyperparameters (such as 1,2 or 3)
#add level 1 train set entity implicit annotations ( hierarchical level 1) into test set
level1_data = set()
final_sc2_1 = set()
level1_data = utils.add_level1_semantic(graph_go, triple_df,second_df,spec_version_v1,entities_v1)
final_sc2_1 = utils.final_test_valid_sc2(triple_df,level1_data,graph_go)

/content/drive/MyDrive/GOA_Dataset/src/utils.py:196: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ).query('_merge == "left_only"').drop('_merge', 1)


In [16]:
#split changed triples in V2 into two: test and valid sets
#1/2 of data to be test and 1/2 valid
test, valid = utils.split_test_valid(final_sc2_1)
#txt file generate
utils.txt_gen(SAVE_PATH+'test_sc2_l1.txt',test)
utils.txt_gen(SAVE_PATH+'valid_sc2_l1.txt',valid)

,0,1,2
0,https://www.uniprot.org/uniprot/X3,http://purl.obolibrary.org/obo/GO_0008150,http://purl.obolibrary.org/obo/g8
1,https://www.uniprot.org/uniprot/X3,http://purl.obolibrary.org/obo/GO_0008150,http://purl.obolibrary.org/obo/g4


**Second test and valid set case:** Level 1 & 2 hierarchical depth of implicit annotations for test and valid sets

In [17]:
#Level 2: Use levels as hyperparameters such as 1,2,3..
#add level 1 and 2 hierarchical implicit annotations into test set
diffv2_v1 = second_df.merge(
    triple_df, how='outer', indicator=True
).query('_merge == "left_only"').drop('_merge', 1)
level2_data = utils.add_level2_semantic(graph_go, diffv2_v1, entities_v1, spec_version_v1)
final_sc2_2 = utils.final_test_valid_sc2(triple_df,level2_data,graph_go)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


In [18]:
#split changed triples in V2 into two: test and valid sets
#1/2 of data to be test and 1/2 valid
test, valid = utils.split_test_valid(final_sc2_2)
#txt file generate
utils.txt_gen(SAVE_PATH+'test_sc2_l2.txt',test)
utils.txt_gen(SAVE_PATH+'valid_sc2_l2.txt',valid)

,0,1,2
0,https://www.uniprot.org/uniprot/X4,http://purl.obolibrary.org/obo/GO_0008150,http://purl.obolibrary.org/obo/g2
1,https://www.uniprot.org/uniprot/X3,http://purl.obolibrary.org/obo/GO_0008150,http://purl.obolibrary.org/obo/g2
2,https://www.uniprot.org/uniprot/X4,http://purl.obolibrary.org/obo/GO_0008150,http://purl.obolibrary.org/obo/g5
